Description : This script truncates and reloads the target identification table 
              (VESSOPS_D.L30_ID.GCI_NCH_PRICING_LFL) from the source BDV satellite table 
              (VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_PRICING_LFL). 
              It standardizes voyage-level competitor pricing data by mapping sailing, voyage, 
              brand, ship, region, cabin, and itinerary attributes. 
              The script also calculates a business key hash (HASH_DIFF) to capture changes, 
              ensuring uniqueness and consistency in downstream processing. 
              Metadata columns (LDTS, RCSR, LASTMODIFIED_BY) are added for lineage and auditing.

Author      : Rakesh J (Mu-Sigma)
Date        : 2025-08-26

Updates     : Nil
Date        : Nil

In [ ]:
# # Import python packages
# import streamlit as st
# import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


## RSS_PRICING_LFL_BDV_ID

In [ ]:
-- create or replace table VESSOPS_D.L30_ID.GCI_NCH_PRICING_LFL as 
TRUNCATE TABLE VESSOPS_D.L30_ID.GCI_NCH_PRICING_LFL;

INSERT INTO VESSOPS_D.L30_ID.GCI_NCH_PRICING_LFL
SELECT
    COMPANY_ID,
    SAIL_ID AS NCLH_SAILING_ID,
    NCLH_UNI_VOYAGE_CODE AS GCI_VOYAGE_CODE,
    BRAND_META_CATEGORY_DESC,
    CRUISELINENAME_LFL AS BRAND_COMPETITOR,
    SOURCESHIPNAME_LFL AS SHIP,
    RSS_COMP_SAIL_ID AS COMP_SAIL_ID,
    NCLH_UNI_VOYAGE_CODE_LFL AS GCI_VOYAGE_CODE_COMPETITOR,
    ADV_WEEK_LFL as ADVANCE_WEEKS_TO_SAIL,
    CabinType_LFL AS ORIGINAL_META,

    /* ✅ Replace NULL TRANSLATED_META with 'UNKNOWN' */
    COALESCE(RSS_CabinType_LFL, 'UNKNOWN') AS TRANSLATED_META,

    TRADE_HIERARCHY1_LFL AS COMP_REGION_1,
    TRADE_HIERARCHY2_LFL AS COMP_REGION_2,
    TRADE_HIERARCHY3_LFL AS COMP_REGION_3,
    CRUISEDURATION_LFL AS DURATION,
    EMBARKATIONDATE_LFL AS DEPARTURE_DATE,
    EMBARKATIONPORT_LFL AS EMBARK_PORT,
    EMBARKATIONPORTUN_LOCODE_LFL AS EMBARK_PORT_UNLOCODE,
    DisembarkationDate_LFL AS DISEMBARK_DATE,
    DISEMBARKATIONPORT_LFL AS DISEMBARK_PORT,
    DISEMBARKATIONPORTUN_LOCODE_LFL AS DISEMBARK_PORT_UNLOCODE,
    TaxExclusiveRatePPD_LFL AS LEAD_TAX_EXCLUSIVE_PRICE_PPD,
    TaxInclusiveRatePPD_LFL AS LEAD_TAX_INCLUSIVE_PRICE_PPD,
    TaxExclusiveRatePP_LFL AS LEAD_TAX_EXCLUSIVE_RATE_PP,
    TaxInclusiveRatePP_LFL AS LEAD_TAX_INCLUSIVE_RATE_PP,
    TAXESANDFEESPPD_LFL AS LEAD_TAX_FEES_PPD,
    TAXESANDFEESPP_LFL AS LEAD_TAX_FEES_PP,

    /* ✅ Replace NULL currency with 'USD' */
    COALESCE(SOURCECURRENCY_LFL, 'USD') AS CURRENCY,

    RateInquiryDate_LFL AS SCRAPE_DATE,
    CACHEPAGELINK_LFL AS CACHEPAGELINK,
    ItineraryPortContent_LFL AS ITINERARY_PORT_CONTENT,
    RSS_MD5_VOYAGE_HASH_KEY,

    /* ✅ Ensure same COALESCE rules applied for HASH_DIFF */
    MD5(
        COALESCE(CAST(COMPANY_ID AS VARCHAR),'') || '|' ||
        COALESCE(CAST(SAIL_ID AS VARCHAR),'') || '|' ||
        COALESCE(NCLH_UNI_VOYAGE_CODE,'') || '|' ||
        COALESCE(CRUISELINENAME_LFL,'') || '|' ||
        COALESCE(SOURCESHIPNAME_LFL,'') || '|' ||
        COALESCE(RSS_COMP_SAIL_ID,'') || '|' ||
        COALESCE(NCLH_UNI_VOYAGE_CODE_LFL,'') || '|' ||
        COALESCE(TRADE_HIERARCHY1_LFL,'') || '|' ||
        COALESCE(TRADE_HIERARCHY2_LFL,'') || '|' ||
        COALESCE(TRADE_HIERARCHY3_LFL,'') || '|' ||
        COALESCE(CAST(CRUISEDURATION_LFL AS VARCHAR),'') || '|' ||
        COALESCE(EMBARKATIONPORT_LFL,'') || '|' ||
        COALESCE(EMBARKATIONPORTUN_LOCODE_LFL,'') || '|' ||
        COALESCE(DISEMBARKATIONPORT_LFL,'') || '|' ||
        COALESCE(DISEMBARKATIONPORTUN_LOCODE_LFL,'') || '|' ||
        COALESCE(CAST(TaxExclusiveRatePPD_LFL AS VARCHAR),'') || '|' ||
        COALESCE(CAST(TaxInclusiveRatePPD_LFL AS VARCHAR),'') || '|' ||
        COALESCE(CAST(TaxExclusiveRatePP_LFL AS VARCHAR),'') || '|' ||
        COALESCE(CAST(TaxInclusiveRatePP_LFL AS VARCHAR),'') || '|' ||
        COALESCE(CAST(TAXESANDFEESPPD_LFL AS VARCHAR),'') || '|' ||
        COALESCE(CAST(TAXESANDFEESPP_LFL AS VARCHAR),'') || '|' ||
        COALESCE(SOURCECURRENCY_LFL, 'USD') || '|' ||              -- ✅ match currency fix
        COALESCE(ItineraryPortContent_LFL,'') || '|' ||
        COALESCE(RSS_CabinType_LFL, 'UNKNOWN')                      -- ✅ match translated meta fix
    ) AS HASH_DIFF,

    current_timestamp as LDTS,
    RCSR,
    current_user as LASTMODIFIED_BY
FROM VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_PRICING_LFL;